In [1]:
from dotenv import load_dotenv
import json
import os
import pandas as pd
import requests

In [2]:
load_dotenv()
EZRF_API_KEY = os.getenv('EZRF_API_KEY')
CHAIN_MAPPINGS = json.load(open('data/chains.json', 'r'))

In [3]:
def fetch_data(limit, cursor):
    url = f'https://ezrf-impact.vercel.app/api/trpc/projects.list?input=%7B%22json%22%3A%7B%22limit%22%3A{limit}%2C%22cursor%22%3A{cursor}%7D%7D'
    headers = {
        'content-type': 'application/json',
        'round-id': 'the-sunnys',
        'x-api-key': EZRF_API_KEY
    }

    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        print(f"Data fetched successfully! (Page {cursor})")
        payload = response.json()
        json_data = payload['result']['data']['json']
        return json_data
    else:
        print(f"Failed to fetch data. Status code: {response.status_code}")

applications = []
est_apps = 2000
lim = 200
curs = 0
while curs * lim < est_apps:
    data = fetch_data(lim, curs)
    if data:
        applications.extend(data)
        curs += 1
    if not data or len(data) < lim:
        break
print(f"Total of {len(applications)} applications fetched.")

Data fetched successfully! (Page 0)
Data fetched successfully! (Page 1)
Data fetched successfully! (Page 2)
Data fetched successfully! (Page 3)
Data fetched successfully! (Page 4)
Data fetched successfully! (Page 5)
Data fetched successfully! (Page 6)
Data fetched successfully! (Page 7)
Total of 1418 applications fetched.


In [4]:
def clean_address(a):
    if not isinstance(a, str):
        return None
    a = a.lower().strip()
    if a[:2] != '0x':
        return None
    return a

def validate_app(addr_type, addr, chain_name, category):
    if category == 'Other Category':
        return "Review - Other Category"
    elif category in ['Channels', 'Frames']:
        return "Review - Farcaster App"
    elif not addr:
        return "Reject - Missing/invalid address"
    elif addr_type == 'mintingWallet':
        if category in ['Art NFTs', 'Other Media NFTs']:
            return "OK"
        else:
            return "Review - Applied with mintingWallet but is not in an NFT category"
    elif addr_type == 'contract':
        if chain_name:
            return "OK"
        else:
            return "Review - Applied with contract but no chainID"
    else:
        return "Review - Did not follow any of the expected applicant pathways"


chain_list = []
normalized_data = []
for (i,app) in enumerate(applications):
    
    profile = app.get('profile', {})
    if not profile:
        profile = {}
    profile_name = profile.get('name', '')
    metadata = app.get('metadata', {})
    awards = metadata.get('sunnyAwards', {})
    project_type = awards.get('projectType', '').title()
    if project_type == 'Other':
        project_type = 'Other Application'
    category = awards.get('category', '')
    if category == 'Other':
        category = 'Other Category'
    contracts = awards.get('contracts', [])    
    
    if len(contracts) > 1:
        print("WARNING: Array encountered at index:", i)
        break
    elif len(contracts) == 1:
        contract = contracts[0]
        address_type = 'contract'
        address = contract.get('address')
        chain_id = contract.get('chainId')
        chain_list.append(chain_id)
        chain = CHAIN_MAPPINGS.get(str(chain_id), 'All Superchain')
    else:
        address_type = 'mintingWallet'
        address = awards.get('mintingWalletAddress')
        chain_id = None
        chain = 'All Superchain'
    address = clean_address(address)
    if not address:
        address_type = 'N/A'
        chain = None
    
    auto_validate = validate_app(
        addr_type=address_type,
        addr=address,
        chain_name=chain,
        category=category
    )

    app_data = {
        'id': app['id'],
        'attester': app['attester'],
        'recipient': app['recipient'],
        'time': app['time'],
        'name': app['name'],
        'schemaId': app['schemaId'],
        'status': app['status'],
        'round': app['round'],
        'profile_name': profile_name,
        'profile_url': f"https://warpcast.com/{profile_name}" if profile_name else '',
        'profile_image': profile.get('profileImageUrl', ''),
        'profile_banner': profile.get('bannerImageUrl', ''),
        'metadata_name': metadata.get('name', ''),
        'metadata_bio': metadata.get('bio', ''),
        'metadata_website': metadata.get('websiteUrl', ''),
        'project_type': project_type,
        'category': category,
        'category_details': awards.get('categoryDetails', ''),
        'avatar_url': awards.get('avatarUrl', ''),
        'cover_image_url': awards.get('coverImageUrl', ''),
        'address_type': address_type,
        'address': address,
        'chain_id': chain_id,
        'chain':  chain,
        'auto_validation': auto_validate,
        'passed_auto_validation': "OK" in auto_validate
    }
    normalized_data.append(app_data)

df = pd.DataFrame(normalized_data)
count_passed_auto_validation = df.passed_auto_validation.sum()
df

,id,attester,recipient,time,name,schemaId,status,round,profile_name,profile_url,...,category,category_details,avatar_url,cover_image_url,address_type,address,chain_id,chain,auto_validation,passed_auto_validation
0,0x66076854e0f9ce49078c76ee39e2e9fae61a8526f406...,0x8Bc704386DCE0C4f004194684AdC44Edf6e85f07,0xE4EE538019673501F4B75de5aF5CC073Ec0A1487,1724860607,0x,0xf8757b1e38ff1b0c1893e47f7d815367332bec28fea4...,approved,0x636d30617975733335303030357a7779623476747572...,ewokafloka,https://warpcast.com/ewokafloka,...,DEX,0x = developer focused APIs for building on De...,https://cdn.charmverse.io/user-content/0273f96...,https://cdn.charmverse.io/user-content/0273f96...,contract,0xdef1c0ded9bec7f1a1670819833240f027b25eff,8453.0,Base,OK,True
1,0x5a587244ecf18246e881cda06bac1ece7c0995bb2faf...,0x8Bc704386DCE0C4f004194684AdC44Edf6e85f07,0x0000000000000000000000000000000000000000,1724723573,1001 raisons,0xf8757b1e38ff1b0c1893e47f7d815367332bec28fea4...,approved,0x636d30617975733335303030357a7779623476747572...,kawz,https://warpcast.com/kawz,...,Art NFTs,,https://cdn.charmverse.io/user-content/648db62...,https://cdn.charmverse.io/user-content/648db62...,mintingWallet,0x8f59aa0f586d6d941152b9075b344cdc42e9b024,NaN,All Superchain,OK,True
2,0xc3d1a57ca75f8416819fd0e7158c85a224a30962ad20...,0x8Bc704386DCE0C4f004194684AdC44Edf6e85f07,0x258a3790639F0dC736eAF5b8817706417C656a47,1725717115,1BITCUDA,0xf8757b1e38ff1b0c1893e47f7d815367332bec28fea4...,pending,0x636d30617975733335303030357a7779623476747572...,cudaoutofmemory,https://warpcast.com/cudaoutofmemory,...,Art NFTs,https://zora.co/@cudaoutofmemory?collection=zo...,https://cdn.charmverse.io/user-content/805b8ff...,https://cdn.charmverse.io/user-content/805b8ff...,mintingWallet,0xd0966b2d8d48e057105afa7a1b84bf1188fb00f0,NaN,All Superchain,OK,True
3,0x9f6404eed4cb04fd2a0513887bb13c0ccc29dc9439cc...,0x8Bc704386DCE0C4f004194684AdC44Edf6e85f07,0x72bA95D9701B0aE496AB4320010a43a764905b1b,1724763747,1 Million Dream,0xf8757b1e38ff1b0c1893e47f7d815367332bec28fea4...,pending,0x636d30617975733335303030357a7779623476747572...,armin,https://warpcast.com/armin,...,Art NFTs,,https://cdn.charmverse.io/user-content/034b136...,https://cdn.charmverse.io/user-content/034b136...,mintingWallet,0x72ba95d9701b0ae496ab4320010a43a764905b1b,NaN,All Superchain,OK,True
4,0x83343ef2cd78008735dd4f7d0de520b8678ee818fac3...,0x8Bc704386DCE0C4f004194684AdC44Edf6e85f07,0xFA1aFC4534fc9F80a552e61Dd04CD8A172c821a6,1724762469,2021go in Web3,0xf8757b1e38ff1b0c1893e47f7d815367332bec28fea4...,pending,0x636d30617975733335303030357a7779623476747572...,jaxo,https://warpcast.com/jaxo,...,Other Category,Web3 guide,https://cdn.charmverse.io/user-content/085011e...,,N/A,None,NaN,None,Review - Other Category,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1413,0x83f84ea97054ed128e47c90800be7e4d9a7f785b2a8f...,0x8Bc704386DCE0C4f004194684AdC44Edf6e85f07,0x6B8288b5793D67eAFc89CA423F6A30b5ACe17dED,1725282971,Zuma,0xf8757b1e38ff1b0c1893e47f7d815367332bec28fea4...,pending,0x636d30617975733335303030357a7779623476747572...,zuma,https://warpcast.com/zuma,...,Staking,,,,N/A,None,NaN,None,Reject - Missing/invalid address,False
1414,0xee53b36d13c1ad16a894d48230ac2923f77acb24e438...,0x8Bc704386DCE0C4f004194684AdC44Edf6e85f07,0xF14f94fF4c8c63Dfa73d82645Ef82743BB298231,1725505273,ZuztantivoZero,0xf8757b1e38ff1b0c1893e47f7d815367332bec28fea4...,pending,0x636d30617975733335303030357a7779623476747572...,moctezuma3ro,https://warpcast.com/moctezuma3ro,...,Identity,Clothing & jewelry,https://cdn.charmverse.io/user-content/8e57f00...,https://cdn.charmverse.io/user-content/8e57f00...,N/A,None,NaN,None,Reject - Missing/invalid address,False
1415,0x51d07a8792a25d8555f0d8145223911020ed99d9b1eb...,0x8Bc704386DCE0C4f004194684AdC44Edf6e85f07,0x8fEEe00f1d8C355086b4667f153C744688b7F6b0,1724765749,ƝЄ†grԼ,0xf8757b1e38ff1b0c1893e47f7d815367332bec28fea4...,pending,0x636d30617975733335303030357a7779623476747572...,guruguruhyena,http

In [5]:
with open("data/applications.json", "w") as f:
    json.dump(applications, f, indent=2)
    
df.drop(columns=[
    'attester', 'schemaId', 'status', 'round', 'profile_image', 'profile_banner',
    'metadata_bio', 'avatar_url', 'cover_image_url', 'category_details'
]).to_csv("data/applications_reviewed.csv")    

In [6]:
df[df['chain_id'].isin([2522,6080])]

,id,attester,recipient,time,name,schemaId,status,round,profile_name,profile_url,...,category,category_details,avatar_url,cover_image_url,address_type,address,chain_id,chain,auto_validation,passed_auto_validation
101,0x816774d46d452fb30e59c371e211125ec0f99c64f030...,0x8Bc704386DCE0C4f004194684AdC44Edf6e85f07,0x6a2E16EA3a608E8D697B3fc0f59d0012A76901e4,1724766129,Balancer,0xf8757b1e38ff1b0c1893e47f7d815367332bec28fea4...,approved,0x636d30617975733335303030357a7779623476747572...,lipman,https://warpcast.com/lipman,...,DEX,,https://storage.googleapis.com/op-atlas/f5afe0...,https://storage.googleapis.com/op-atlas/18fb1c...,contract,0xba12222222228d8ba445958a75a0704d566bf2c8,2522.0,All Superchain,OK,True
988,0xd855e5b06a1a03a5cc4d3e75dcce7c1efb5e7f8f8c45...,0x8Bc704386DCE0C4f004194684AdC44Edf6e85f07,0x0000000000000000000000000000000000000000,1725730621,Owlto Finance,0xf8757b1e38ff1b0c1893e47f7d815367332bec28fea4...,approved,0x636d30617975733335303030357a7779623476747572...,kawz,https://warpcast.com/kawz,...,Bridges,,https://storage.googleapis.com/op-atlas/a7ef0d...,https://storage.googleapis.com/op-atlas/858356...,contract,0xc626845bf4e6a5802ef774da0b3dfc6707f015f7,2522.0,All Superchain,OK,True
989,0xf0aaeef07e6674aa6edb6d9b0caa7e9386ebf7576b96...,0x8Bc704386DCE0C4f004194684AdC44Edf6e85f07,0x0000000000000000000000000000000000000000,1725730801,Owlto Finance,0xf8757b1e38ff1b0c1893e47f7d815367332bec28fea4...,approved,0x636d30617975733335303030357a7779623476747572...,kawz,https://warpcast.com/kawz,...,Bridges,,https://storage.googleapis.com/op-atlas/a7ef0d...,https://storage.googleapis.com/op-atlas/858356...,contract,0xc626845bf4e6a5802ef774da0b3dfc6707f015f7,6080.0,All Superchain,OK,True
